# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name: Kayla

#### [Github Repo] (https://github.com/kcbreum/json-sentiment)

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

In [13]:
import requests
import json
import pickle
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

print('All prereqs installed')
!pip list

All prereqs installed
Package            Version
------------------ -----------
annotated-types    0.6.0
asttokens          2.4.1
blis               0.7.11
catalogue          2.0.10
certifi            2024.2.2
charset-normalizer 3.3.2
click              8.1.7
cloudpathlib       0.16.0
colorama           0.4.6
comm               0.2.2
confection         0.1.4
cymem              2.0.8
debugpy            1.8.1
decorator          5.1.1
en-core-web-sm     3.7.1
executing          2.0.1
idna               3.6
ipykernel          6.29.4
ipython            8.23.0
jedi               0.19.1
Jinja2             3.1.3
joblib             1.3.2
jupyter_client     8.6.1
jupyter_core       5.7.2
langcodes          3.3.0
MarkupSafe         2.1.5
matplotlib-inline  0.1.6
murmurhash         1.0.10
nest-asyncio       1.6.0
nltk               3.8.1
numpy              1.26.4
packaging          24.0
parso              0.8.3
pip                24.0
platformdirs       4.2.0
preshed            3.0.9
prompt-toolki

### Question 1

1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [14]:
AUTHOR='Edgar Allan Poe'
POEM = 'A Dream Within A Dream'

#only certain poets and titles are available
#to see the available poets, go to (in a web browser)
# https://poetrydb.org/author
#To see which poems that author has available, go to 
# https://poetrydb.org/author/AUTHOR NAME
# e.g.: https://poetrydb.org/author/Edgar Allan Poe
#The spaces will get handled by your web browser

# A cool pythonism (introduced in Python 3): f strings
# https://docs.python.org/3/tutorial/inputoutput.html#tut-f-strings
URL = f'https://poetrydb.org/author,title/{AUTHOR};{POEM}'
result = json.loads(requests.get(URL).text)
poem = '\n'.join(result[0]['lines']) 

with open('poem.json', 'w') as file:
    json.dump(result, file)

### Question 2

2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [15]:
with open('poem.json', 'r') as file:
    result = json.load(file)

poem = '\n'.join(result[0]['lines'])
print(poem)

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

doc = nlp(poem)

polarity_score = doc._.polarity

print(f"Polarity Score: {polarity_score}")

Take this kiss upon the brow!
And, in parting from you now,
Thus much let me avow--
You are not wrong, who deem
That my days have been a dream:
Yet if hope has flown away
In a night, or in a day,
In a vision or in none,
Is it therefore the less _gone_?
_All_ that we see or seem
Is but a dream within a dream.

I stand amid the roar
Of a surf-tormented shore,
And I hold within my hand
Grains of the golden sand--
How few! yet how they creep
Through my fingers to the deep
While I weep--while I weep!
O God! can I not grasp
Them with a tighter clasp?
O God! can I not save
_One_ from the pitiless wave?
Is _all_ that we see or seem
But a dream within a dream?
Polarity Score: 0.055555555555555546


### Question 3

3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [16]:
import requests
import json

def get_lyrics(artist, song, filename):
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
    response = requests.get(url)
    if response.status_code == 200:
        data = json.loads(response.content)
        if 'lyrics' in data:
            lyrics = data['lyrics']
            with open(filename, 'w') as file:
                file.write(lyrics)
            
            print(f"Lyrics saved to {filename}")
        else:
            print("Lyrics not found for the specified song.")
    else:
        print("Error fetching lyrics. Check your input or try again later.")

get_lyrics("Ed Sheeran", "Shape of You", "shape_of_you_lyrics.txt")
get_lyrics("Taylor Swift", "Love Story", "love_story_lyrics.txt")
get_lyrics("Adele", "Hello", "hello_lyrics.txt")
get_lyrics("Queen", "Bohemian Rhapsody", "bohemian_rhapsody_lyrics.txt")

Lyrics saved to shape_of_you_lyrics.txt
Lyrics saved to love_story_lyrics.txt
Lyrics saved to hello_lyrics.txt
Lyrics saved to bohemian_rhapsody_lyrics.txt


### Question 4

4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [18]:
from textblob import TextBlob

def get_polarity_score(filename):
    with open(filename, 'r') as file:
        lyrics = file.read()

    blob = TextBlob(lyrics)
    polarity_score = blob.sentiment.polarity
    return polarity_score

shape_of_you_score = get_polarity_score("shape_of_you_lyrics.txt")
love_story_score = get_polarity_score("love_story_lyrics.txt")
hello_score = get_polarity_score("hello_lyrics.txt")
bohemian_rhapsody_score = get_polarity_score("bohemian_rhapsody_lyrics.txt")

print("Polarity Scores:")
print("Shape of You:", shape_of_you_score)
print("Love Story:", love_story_score)
print("Hello:", hello_score)
print("Bohemian Rhapsody:", bohemian_rhapsody_score)

Polarity Scores:
Shape of You: 0.32200413223140495
Love Story: 0.06875
Hello: -0.15366666666666665
Bohemian Rhapsody: -0.0029495701058200783


Some of these scores, such as Shape of You and Hello, make sense. Shape of You has a pretty positive connotation, and Hello has very sad wordings. Similarly, Love Story is a mix of being upset at the forbidden love but at the same time has all the wording of a happy love song, especially by the end, so it makes sense for its score to be fairly close to a 0. The one I'm most suprised by is Bohemian Rhapsody. When looking at the lyrics, there aren't really very many positives to the story it tells, so I assumed it would have a lower negative score.